# Gender Wage Gap Data Processing

In [2]:
import pandas as pd
import numpy as np

## 1 Map Country Codes

In [4]:
# numeric is country id, which will be used in the map to identify countries
df_country = pd.read_excel("country_codes.xlsx")
df_country.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Åland Islands,AX,ALA,248
1,Zimbabwe,ZW,ZWE,716
2,Zambia,ZM,ZMB,894
3,Yemen,YE,YEM,887
4,Western Sahara,EH,ESH,732


In [27]:
df_gap = pd.read_csv("gap.csv", delimiter=";")
df_gap.head()

,location,indicator,subject,measure,frequency,time,value
0,AUS,WAGEGAP,EMPLOYEE,PC,A,1975,21.582.733.813
1,AUS,WAGEGAP,EMPLOYEE,PC,A,1976,20.754.716.981
2,AUS,WAGEGAP,EMPLOYEE,PC,A,1977,18.390.804.598
3,AUS,WAGEGAP,EMPLOYEE,PC,A,1978,19.791.666.667
4,AUS,WAGEGAP,EMPLOYEE,PC,A,1979,20


In [28]:
df = df_gap.merge(df_country[["Alpha-3 code", "Country", "Numeric"]], left_on='location', right_on='Alpha-3 code', how='left')
df.head()

,location,indicator,subject,measure,frequency,time,value,Alpha-3 code,Country,Numeric
0,AUS,WAGEGAP,EMPLOYEE,PC,A,1975,21.582.733.813,AUS,Australia,36.0
1,AUS,WAGEGAP,EMPLOYEE,PC,A,1976,20.754.716.981,AUS,Australia,36.0
2,AUS,WAGEGAP,EMPLOYEE,PC,A,1977,18.390.804.598,AUS,Australia,36.0
3,AUS,WAGEGAP,EMPLOYEE,PC,A,1978,19.791.666.667,AUS,Australia,36.0
4,AUS,WAGEGAP,EMPLOYEE,PC,A,1979,20,AUS,Australia,36.0


In [29]:
# 'OECD', 'EU27' can't match any country code
df[df["Alpha-3 code"].isna()]["location"].unique()

array(['OECD', 'EU27'], dtype=object)

In [32]:
EU27_str = "Austria, Belgium, Bulgaria, Croatia, Cyprus, the Czech Republic, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, the Netherlands, Poland, Portugal, Romania, Slovak Republic, Slovenia, Spain, Sweden"
EU27_countries = EU27_str.split(", ")
assert(len(EU27_countries) == 27)
all_countries = df["Country"].unique()
print("Among EU27 countries, the following countries don't have its own data:")
for country in EU27_countries:
    if (country not in all_countries):
        print(country)

Among EU27 countries, the following countries don't have its own data:
the Czech Republic
the Netherlands
Slovak Republic


We won't draw EU27 and OECD countries in the map, but we'll draw it in the bar chart!

## 2 Data Processing

In [33]:
# edit wierd value columns
df["split_value"] = df["value"].str.split(".")
df["joined_value"] = df["split_value"].apply(lambda x: ".".join(x[:2]))
df["value"] = df["joined_value"].astype(float)
df = df.drop(["split_value", "joined_value"], axis=1)
df

,location,indicator,subject,measure,frequency,time,value,Alpha-3 code,Country,Numeric
0,AUS,WAGEGAP,EMPLOYEE,PC,A,1975,21.582,AUS,Australia,36.0
1,AUS,WAGEGAP,EMPLOYEE,PC,A,1976,20.754,AUS,Australia,36.0
2,AUS,WAGEGAP,EMPLOYEE,PC,A,1977,18.390,AUS,Australia,36.0
3,AUS,WAGEGAP,EMPLOYEE,PC,A,1978,19.791,AUS,Australia,36.0
4,AUS,WAGEGAP,EMPLOYEE,PC,A,1979,20.000,AUS,Australia,36.0
...,...,...,...,...,...,...,...,...,...,...
1205,EU27,WAGEGAP,EMPLOYEE,PC,A,2016,115.776,NaN,NaN,NaN
1206,EU27,WAGEGAP,EMPLOYEE,PC,A,2017,113.314,NaN,NaN,NaN
1207,EU27,WAGEGAP,EMPLOYEE,PC,A,2018,111.385,NaN,NaN,NaN
1208,EU27,WAGEGAP,EMPLOYEE,PC,A,2019,111.223,NaN,NaN,NaN


In [34]:
df["subject"].value_counts()

subject
EMPLOYEE        766
SELFEMPLOYED    444
Name: count, dtype: int64

In [61]:
df_employee = df[df["subject"] == "EMPLOYEE"]
df_employee
# drop OECD and EU27
# df_employee_id = df_employee.dropna(subset=['Numeric'])
# df_employee_id["Numeric"] = df_employee_id["Numeric"].astype('int')
# df_employee_id
# df_employee_id_out = pd.pivot_table(df_employee_id, values='value', index=['time'], columns=['Numeric'])

,location,indicator,subject,measure,frequency,time,value,Alpha-3 code,Country,Numeric
0,AUS,WAGEGAP,EMPLOYEE,PC,A,1975,21.582,AUS,Australia,36.0
1,AUS,WAGEGAP,EMPLOYEE,PC,A,1976,20.754,AUS,Australia,36.0
2,AUS,WAGEGAP,EMPLOYEE,PC,A,1977,18.390,AUS,Australia,36.0
3,AUS,WAGEGAP,EMPLOYEE,PC,A,1978,19.791,AUS,Australia,36.0
4,AUS,WAGEGAP,EMPLOYEE,PC,A,1979,20.000,AUS,Australia,36.0
...,...,...,...,...,...,...,...,...,...,...
1205,EU27,WAGEGAP,EMPLOYEE,PC,A,2016,115.776,NaN,NaN,NaN
1206,EU27,WAGEGAP,EMPLOYEE,PC,A,2017,113.314,NaN,NaN,NaN
1207,EU27,WAGEGAP,EMPLOYEE,PC,A,2018,111.385,NaN,NaN,NaN
1208,EU27,WAGEGAP,EMPLOYEE,PC,A,2019,111.223,NaN,NaN,NaN


In [82]:
output = {} # {year:{country: {id:, name:, value:,}}}
for _, row in df_employee.iterrows():
    year = row['time']
    country = row['location']
    id = row['Numeric']
    value = row['value']
    name = country if country in ['OECD', 'EU27'] else row['Country']
    if year not in output:
        output[year] = {}
    output[year][country] = {}
    output[year][country]['id'] = int(id) if not pd.isna(id) else None
    output[year][country]['name'] = name
    output[year][country]['value'] = value

In [84]:
import json
with open("employee_wage_gap.json", "w") as outfile:
    json.dump(output, outfile)
json.dumps(output)

'{"1975": {"AUS": {"id": 36, "name": "Australia", "value": 21.582}, "JPN": {"id": 392, "name": "Japan", "value": 42.389}, "GBR": {"id": 826, "name": "United Kingdom of Great Britain and Northern Ireland (the)", "value": 39.855}, "USA": {"id": 840, "name": "United States of America (the)", "value": 3.763}}, "1976": {"AUS": {"id": 36, "name": "Australia", "value": 20.754}, "JPN": {"id": 392, "name": "Japan", "value": 39.741}, "GBR": {"id": 826, "name": "United Kingdom of Great Britain and Northern Ireland (the)", "value": 36.55}, "USA": {"id": 840, "name": "United States of America (the)", "value": 37.755}}, "1977": {"AUS": {"id": 36, "name": "Australia", "value": 18.39}, "FIN": {"id": 246, "name": "Finland", "value": 2.769}, "JPN": {"id": 392, "name": "Japan", "value": 40.326}, "GBR": {"id": 826, "name": "United Kingdom of Great Britain and Northern Ireland (the)", "value": 3.578}, "USA": {"id": 840, "name": "United States of America (the)", "value": 38.224}}, "1978": {"AUS": {"id": 36,

In [86]:
df_employee.describe()

,time,value,Numeric
count,766.000000,766.000000,721.000000
mean,2005.124021,38.265646,422.061026
std,11.176961,41.167896,260.182327
min,1970.000000,0.384387,36.000000
25%,1999.000000,15.802500,208.000000
50%,2007.000000,21.852500,392.000000
75%,2014.000000,42.004750,620.000000
max,2020.000000,419.340000,840.000000


In [80]:
df_employee_out.to_json("employee_json.json")
df_selfemployed_out.to_json("selfemployed_json.json")

In [84]:
df_employee_out.to_csv("employee_data.csv")
df_selfemployed_out.to_csv("selfemployed_data.csv")

In [86]:
df_employee_out_2 = pd.pivot_table(df_employee, values='value', index=['location'], columns=['time'])
df_employee_out_2

time,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
location,,,,,,,,,,,,,,,,,,,,,
AUS,NaN,NaN,NaN,NaN,NaN,21.582,20.754,18.390,19.791,20.000,...,15.966,13.750000,18.000,17.050,15.384,13.728,14.507,16.142,14.965,12.266
AUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.550,18.185000,18.053,17.728,1.704,15.670,1.538,14.876,14.011,13.325
BEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58.381,64.111000,59.128,32.989,4.672,37.012,41.902,34.087,3.798,NaN
BGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,40.625,NaN,NaN,NaN,30.303,33.577,25.546
CAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.175,19.511000,1.930,19.229,1.862,18.221,1.817,18.518,17.607,16.112
CHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,19.343000,NaN,16.940,NaN,14.762,NaN,15.095,17.163,13.783
CHL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.000,NaN,10.666,NaN,14.285,NaN,12.500,NaN,NaN,85.964
COL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.348,81.979000,71.428,9.600,11.141,70.917,76.923,57.879,4.000,NaN
CRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.306,0.552486,NaN,51.497,37.037,17.857,2.967,47.252,NaN,NaN


In [97]:
df_employee_out_2.to_json("employee_time.json")

In [98]:
df_employee_out_2.to_csv("employee_time.csv")